In [44]:
import numpy as np
import vtktools
import torch
import time 
from tqdm import tqdm
from scipy.sparse import csr_matrix

In [45]:
def get_edge_list_for_node(input, vtu_graph):
    
    node_neighbours = vtu_graph.GetPointPoints(input)
    node_edge_list = []
    
    for node in node_neighbours:
        if node > input:  # change to >= to include the same node
            node_edge_list.append([input, node])
            
    return node_edge_list

def get_complete_edge_list(vtu_object, n_points):

    complete_edge_list = [] 
    #complete_edge_list = csr_matrix((n_points, n_points))
    for point in tqdm(range(n_points)):  #
        #temp = get_edge_list_for_node(point, vtu_object)
        #d2 = [item[1] for item in temp]
        #print(d2)
        complete_edge_list = complete_edge_list + get_edge_list_for_node(point, vtu_object)
        #complete_edge_list[point, d2] = temp
    complete_edge_list = np.array(complete_edge_list)
    # complete_edge_list = torch.tensor(complete_edge_list).t()
        
    return complete_edge_list

In [37]:
filename = '../../WaveSuite/regularWave/regularWave_1.vtu'
#First object
first_object = vtktools.vtu(filename) 

In [47]:
#This two parameters are the same for every graph, so they would be calculated just once instead of in every iteration
features_list = ['nut'] 

n_points = first_object.GetField(features_list[0]).shape[0]
n_points 

311203

In [48]:
start = time.time()
n_points = n_points
complete_edge_list = get_complete_edge_list(first_object, n_points)

end = time.time()

print(end-start)

100%|██████████| 311203/311203 [1:51:39<00:00, 46.45it/s]  

6699.825669050217


In [50]:
filename = 'wave_complete_edge_list'
np.save(filename+'.npy',complete_edge_list)


In [52]:
complete_edge_list.shape

(3738056, 2)

In [53]:
start = time.time()

complete_edge_list = torch.tensor(complete_edge_list).t()

end = time.time()

print(end-start)

0.021915674209594727


In [54]:
torch.save(complete_edge_list, filename+'.pt')